# **MIP3-Run1**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.5 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']



### **Generate Mathematical Model**

In [4]:
problem = """You are tasked with scheduling the power output of 6 electric power thermal units over the timespan of 15 periods. There is a constant startup cost of for each power unit that is applied if the power plant is turned on. The startup costs are 10324€, 5678€, 7802€, 12899€, 4596€ and 9076€ for powerplants 1 to 6, respectively. In addition, there is a constant shutdown cost for each power unit that is applied if the power plant is turned off. The shutdown costs are 2673€, 5893€, 982€, 6783€, 2596€ and 3561€ for powerplants 1 to 6, respectively. There is also a fixed and variable cost applied if the power plant is running. The fixed cost is constant and the variable cost is proportional to the output of a power plant. There are lower and upper bounds for the output power for each unit. The variable costs are different for each power plant and at each time step. Next, there are maximum power increments and decrements for each power plant that limit how much the output can change from one time period to the next. There is a total power demand that needs to be fulfilled by the power units. Finally, for security reasons, the total available power out should always be 10% higher than the demand. """

In [6]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 1
)


In [7]:
print(response1.choices[0].message.content)

Mathematical Model:

Parameters:
- i = number of power plants (i=1,2,...,6)
- t = time periods (t=1,2,...,15)
- S_i = startup costs for power plant i (€)
- Sh_i = shutdown costs for power plant i (€)
- F_i = fixed costs for power plant i (€)
- V_i(t) = variable cost for power plant i at time t (€/MW)
- L_i = lower bound for the output power for power plant i (MW)
- U_i = upper bound for the output power for power plant i (MW)
- R_i = maximum change in power output for power plant i from one time period to the next (MW)
- D(t) = power demand at time t (MW)
- All costs are in Euro (€), power is in Megawatts (MW).

Decision Variables:
- X_i(t) = output of power plant i at time t (MW)
- Z_i(t) = whether power plant i starts up at time t (Binary, 1 if starts, 0 otherwise)
- Y_i(t) = whether power plant i shuts down at time t (Binary, 1 if shuts, 0 otherwise)

Objective Function:
Minimize the total cost over all power plants and time periods which is the sum of startup costs, shutdown costs,

### **Generate Pyomo Code**

In [8]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [10]:
Markdown(response2.choices[0].message.content)

Here is a sample Python Pyomo code that demonstrates how to model this optimization problem. However, the last constraint which requires that the power plant starts working based on Z_i(t) variable and stops workers based on Y_i(t) variable is not explicitly added as it is a logical constraint and pyomo does not provide a straightforward way to represent such constraint.

```python
from pyomo.environ import *

model = ConcreteModel()

# parameters
power_plants = range(1, 7)
timing = range(1, 16)

# sample data (random numbers for demonstration)
S_i = {i: 100 for i in power_plants}
Sh_i = {i: 100 for i in power_plants}
F_i = {i: 100 for i in power_plants}
V_i = {(i,t): 10 for i in power_plants for t in timing}
L_i = {i: 200 for i in power_plants}
U_i = {i: 500 for i in power_plants}
R_i = {i: 150 for i in power_plants}
D_t = {t: 1000 for t in timing}

# variables
model.X = Var(power_plants, timing, domain=NonNegativeReals)
model.Z = Var(power_plants, timing, domain=Binary)
model.Y = Var(power_plants, timing, domain=Binary)

# objective function
model.obj = Objective(expr = sum(
    model.Z[i,t]*S_i[i] + model.Y[i,t]*Sh_i[i] + F_i[i] + model.X[i,t]*V_i[i,t]
    for i in power_plants for t in timing), sense=minimize)

# constraints
model.power_output_constr = ConstraintList()
for i in power_plants:
    for t in timing:
        model.power_output_constr.add(expr = L_i[i] <= model.X[i,t] <= U_i[i])

model.power_increment_constr = ConstraintList()
for i in power_plants:
    for t in timing[1:]:
        model.power_increment_constr.add(expr = -R_i <= model.X[i,t] - model.X[i,t-1] <= R_i[i])

model.power_demand_constr = ConstraintList()
for t in timing:
    model.power_demand_constr.add(expr = D_t[t] <= sum(model.X[i,t] for i in power_plants))

model.security_constr = ConstraintList()
for t in timing:
    model.security_constr.add(expr = sum(model.X[i,t] for i in power_plants) >= 1.1 * D_t[t])

```
In this sample code, I have used random numbers for initializing the values in the parameters dictionary. This wouldn't occur in a real situation. In a real situation, these values would likely be taken from a data source of some kind, perhaps a database or a configuration file. Furthermore, I didn't represent the constraint X_i(1) = 0 if the respective power plant doesn't start at t=1, you may need some additional data and constraints to incorporate this.

### **Run the code Generated by GPT4**

In [11]:
from pyomo.environ import *

model = ConcreteModel()

# parameters
power_plants = range(1, 7)
timing = range(1, 16)

# sample data (random numbers for demonstration)
S_i = {i: 100 for i in power_plants}
Sh_i = {i: 100 for i in power_plants}
F_i = {i: 100 for i in power_plants}
V_i = {(i,t): 10 for i in power_plants for t in timing}
L_i = {i: 200 for i in power_plants}
U_i = {i: 500 for i in power_plants}
R_i = {i: 150 for i in power_plants}
D_t = {t: 1000 for t in timing}

# variables
model.X = Var(power_plants, timing, domain=NonNegativeReals)
model.Z = Var(power_plants, timing, domain=Binary)
model.Y = Var(power_plants, timing, domain=Binary)

# objective function
model.obj = Objective(expr = sum(
    model.Z[i,t]*S_i[i] + model.Y[i,t]*Sh_i[i] + F_i[i] + model.X[i,t]*V_i[i,t]
    for i in power_plants for t in timing), sense=minimize)

# constraints
model.power_output_constr = ConstraintList()
for i in power_plants:
    for t in timing:
        model.power_output_constr.add(expr = L_i[i] <= model.X[i,t] <= U_i[i])

model.power_increment_constr = ConstraintList()
for i in power_plants:
    for t in timing[1:]:
        model.power_increment_constr.add(expr = -R_i <= model.X[i,t] - model.X[i,t-1] <= R_i[i])

model.power_demand_constr = ConstraintList()
for t in timing:
    model.power_demand_constr.add(expr = D_t[t] <= sum(model.X[i,t] for i in power_plants))

model.security_constr = ConstraintList()
for t in timing:
    model.security_constr.add(expr = sum(model.X[i,t] for i in power_plants) >= 1.1 * D_t[t])

PyomoException: Cannot convert non-constant Pyomo expression (200  <=  X[1,1]) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y in [m.x, m.y]:
    ...     pass
would both cause this exception.

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**

In [23]:
from pyomo.environ import *

model = ConcreteModel()

# parameters
power_plants = range(1, 7)
timing = range(1, 16)


# sample data inputted by human
S_i = {1: 10324, 2:5678, 3:7802, 4:12899, 5:4596, 6:9076}
Sh_i = {1:2673, 2:5893, 3: 982, 4:6783, 5:2596, 6:3561}
F_i = {1: 2000, 2: 3000, 3: 2500, 4: 4000, 5: 3500, 6: 4500}
V_i = {(1, 1): 20, (1, 2): 22, (1, 3): 23, (1, 4): 24, (1, 5): 25, (1, 6): 26, (1, 7): 27, (1, 8): 28, (1, 9): 29, (1, 10): 30, (1, 11): 31, (1, 12): 32, (1, 13): 33, (1, 14): 34, (1, 15): 35, (2, 1): 15, (2, 2): 16, (2, 3): 17, (2, 4): 18, (2, 5): 19, (2, 6): 20, (2, 7): 21, (2, 8): 22, (2, 9): 23, (2, 10): 24, (2, 11): 25, (2, 12): 26, (2, 13): 27, (2, 14): 28, (2, 15): 29, (3, 1): 18, (3, 2): 19, (3, 3): 20, (3, 4): 21, (3, 5): 22, (3, 6): 23, (3, 7): 24, (3, 8): 25, (3, 9): 26, (3, 10): 27, (3, 11): 28, (3, 12): 29, (3, 13): 30, (3, 14): 31, (3, 15): 32, (4, 1): 25, (4, 2): 26, (4, 3): 27, (4, 4): 28, (4, 5): 29, (4, 6): 30, (4, 7): 31, (4, 8): 32, (4, 9): 33, (4, 10): 34, (4, 11): 35, (4, 12): 36, (4, 13): 37, (4, 14): 38, (4, 15): 39, (5, 1): 22, (5, 2): 23, (5, 3): 24, (5, 4): 25, (5, 5): 26, (5, 6): 27, (5, 7): 28, (5, 8): 29, (5, 9): 30, (5, 10): 31, (5, 11): 32, (5, 12): 33, (5, 13): 34, (5, 14): 35, (5, 15): 36, (6, 1): 30, (6, 2): 31, (6, 3): 32, (6, 4): 33, (6, 5): 34, (6, 6): 35, (6, 7): 36, (6, 8): 37, (6, 9): 38, (6, 10): 39, (6, 11): 40, (6, 12): 41, (6, 13): 42, (6, 14): 43, (6, 15): 44} # Sample data
L_i = {1: 50, 2: 40, 3: 30, 4: 60, 5: 55, 6: 65}
U_i = {1: 500, 2:  600, 3: 550, 4: 700, 5: 650, 6: 750}
R_i = {i: 150 for i in power_plants} #no difference between ramp up and ramp down. so the necessary data can't be inputted.
D_t = {t: 1000 for t in timing}

# variables
model.X = Var(power_plants, timing, domain=NonNegativeReals)
model.Z = Var(power_plants, timing, domain=Binary)
model.Y = Var(power_plants, timing, domain=Binary)

# objective function
model.obj = Objective(expr = sum(
    model.Z[i,t]*S_i[i] + model.Y[i,t]*Sh_i[i] + F_i[i] + model.X[i,t]*V_i[i,t]
    for i in power_plants for t in timing), sense=minimize)

# constraints
model.power_output_constr = ConstraintList()
for i in power_plants:
    for t in timing:
        model.power_output_constr.add(expr = L_i[i] <= model.X[i,t])

model.power_output_constr1 = ConstraintList()
for i in power_plants:
    for t in timing:
        model.power_output_constr1.add(expr = model.X[i,t] <= U_i[i])

model.power_increment_constr = ConstraintList()
for i in power_plants:
    for t in timing[1:]:
        model.power_increment_constr.add(expr = -R_i >= model.X[i,t] - model.X[i,t-1])

model.power_increment_constr1 = ConstraintList()
for i in power_plants:
    for t in timing[1:]:
        model.power_increment_constr1.add(expr = model.X[i,t] - model.X[i,t-1] <= R_i[i])

model.power_demand_constr = ConstraintList()
for t in timing:
    model.power_demand_constr.add(expr = D_t[t] <= sum(model.X[i,t] for i in power_plants))

model.security_constr = ConstraintList()
for t in timing:
    model.security_constr.add(expr = sum(model.X[i,t] for i in power_plants) >= 1.1 * D_t[t])

TypeError: bad operand type for unary -: 'dict'